In [6]:
!wget https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/Pesos_y_modelos/model2_lstm256_dropout03.keras
!wget https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/Pesos_y_modelos/model3_lstm256x2_dropout03_lstm128.keras
!wget https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/params_quijote/char2idx.pkl
!wget https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/params_quijote/idx2char.pkl
!wget https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/params_quijote/params.pkl

--2025-06-30 03:33:45--  https://github.com/IJozk/DL/raw/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/Pesos_y_modelos/model2_lstm256_dropout03.keras
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IJozk/DL/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/Pesos_y_modelos/model2_lstm256_dropout03.keras [following]
--2025-06-30 03:33:45--  https://raw.githubusercontent.com/IJozk/DL/99d1bebb91fd554f1baf97eacf7eaab8d9bb32fc/src/Pesos_y_modelos/model2_lstm256_dropout03.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4122064 (3.9M) [application/octet-stream]
Saving to: ‘model2_lstm256_dropout03.ker

In [7]:
import tensorflow as tf
import pickle
import numpy as np

In [8]:
model = tf.keras.models.load_model("/content/model2_lstm256_dropout03.keras")
model1 = tf.keras.models.load_model("/content/model3_lstm256x2_dropout03_lstm128.keras")

with open("char2idx.pkl", "rb") as f:
    char2idx = pickle.load(f)

with open("idx2char.pkl", "rb") as f:
    idx2char = pickle.load(f)

with open("params.pkl", "rb") as f:
    params = pickle.load(f)

SEQ_LENGTH = params["SEQ_LENGTH"]
NUM_CHARS = params["NUM_CHARS"]

In [9]:
def sample(preds, temperature=0.2):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generar_texto(seed_text, model, length=100, temperature=0.2):
    generated = seed_text
    # Convert seed_text to lowercase for dictionary lookup
    input_indices = [char2idx[c] for c in seed_text.lower()]

    for _ in range(length):
        x_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS))
        # Use only the last SEQ_LENGTH characters for prediction input
        for t, idx in enumerate(input_indices[-SEQ_LENGTH:]):
            x_pred[0, t, idx] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx2char[next_index]

        generated += next_char
        input_indices.append(next_index)
        # Keep input_indices at the length of SEQ_LENGTH
        input_indices = input_indices[-SEQ_LENGTH:]


    return generated

In [10]:
def preparar_seed(seed, seq_length=40):
    # Rellena con espacios a la derecha si es más corto, o corta si es más largo
    seed = seed.ljust(seq_length)[:seq_length]
    return seed

In [11]:
seed_original = "En un lugar de la Mancha, de cuyo nombre"
seed = preparar_seed(seed_original, 40)

print("Texto generado con el modelo: lstm(128) y dropout de 0.3")
print(generar_texto(seed, model, length=100, temperature=0.2))
print("\n\nTexto generado con el modelo: lstm(256), lstm(256), dropout de 0.3 y lstm(128)")
print(generar_texto(seed, model1, length=100, temperature=0.2))

Texto generado con el modelo: lstm(128) y dropout de 0.3
En un lugar de la Mancha, de cuyo nombre, y así es muyo verdaderon en la mitad de mi amo, que se la entraba en el alma, por lo que se le ha 


Texto generado con el modelo: lstm(256), lstm(256), dropout de 0.3 y lstm(128)
En un lugar de la Mancha, de cuyo nombre, de buena caballería sancho a su reyón y de mi cuenta de diez triado y mejor de don quijote, con mu
